<a href="https://colab.research.google.com/github/eunoia-code/pneumonia-ipynb/blob/main/Pneumonia_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# !ls "/content/drive/My Drive"

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/Dataset_penelitian_pneumonia/preprocess"

testing  training  validasi


In [ ]:
base_path = '/content/drive/My Drive/Dataset_penelitian_pneumonia/preprocess'

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout

In [ ]:
# Load the ResNet152 model
base_model = ResNet152(weights=None, include_top=False, input_shape=(256, 256, 3))

In [ ]:
# Add dropout layer
x = base_model.output
x = Dropout(0.5)(x)  # Dropout rate of 50%
x = tf.keras.layers.GlobalAveragePooling2D()(x)

In [ ]:
# Add final dense layer
predictions = tf.keras.layers.Dense(3, activation='softmax')(x)

In [ ]:
from tensorflow.keras.optimizers import Adam

# Set the learning rate
learning_rate = 0.001

# Initialize the optimizer
optimizer = Adam(learning_rate=learning_rate)

In [ ]:
# Compile the model
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
# Compile the model with the optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    rotation_range=20,
    zoom_range=0.2,
    vertical_flip=True)
test_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    rotation_range=20,
    zoom_range=0.2,
    vertical_flip=True)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    rotation_range=20,
    zoom_range=0.2,
    vertical_flip=True)

In [ ]:
train_generator = train_datagen.flow_from_directory(
        base_path+'/training',  # this is the target directory
        target_size=(224, 224),
        color_mode='grayscale',
        batch_size=32,
        class_mode='categorical')

Found 8188 images belonging to 3 classes.


In [ ]:
validation_generator = val_datagen.flow_from_directory(
        base_path+'/validasi',
        target_size=(224, 224),
        color_mode='grayscale',
        batch_size=32,
        class_mode='categorical')

Found 1515 images belonging to 3 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
        base_path+'/testing',
        target_size=(224, 224),
        color_mode='grayscale',
        batch_size=32,
        class_mode='categorical')

Found 1515 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Initialize checkpoint
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
early = EarlyStopping(monitor="val_loss", mode="min", patience=10)

callbacks = [checkpoint, early]

# Train the model with checkpoint
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples // train_generator.batch_size,
      epochs=25,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples // validation_generator.batch_size,
      callbacks=callbacks)

Epoch 1/25
511/511 [==============================] - 3921s 7s/step - loss: 0.7212 - accuracy: 0.7051 - val_loss: 1.8233 - val_accuracy: 0.3637
Epoch 2/25
511/511 [==============================] - 269s 525ms/step - loss: 0.5862 - accuracy: 0.7471 - val_loss: 1.6720 - val_accuracy: 0.2274
Epoch 3/25
511/511 [==============================] - 264s 517ms/step - loss: 0.5861 - accuracy: 0.7429 - val_loss: 2.2034 - val_accuracy: 0.3378
Epoch 4/25
511/511 [==============================] - 264s 517ms/step - loss: 0.5215 - accuracy: 0.7768 - val_loss: 53.4374 - val_accuracy: 0.3331
Epoch 5/25
511/511 [==============================] - 264s 516ms/step - loss: 0.5400 - accuracy: 0.7698 - val_loss: 2.0007 - val_accuracy: 0.4029
Epoch 6/25
511/511 [==============================] - 264s 516ms/step - loss: 0.5042 - accuracy: 0.7901 - val_loss: 2.5699 - val_accuracy: 0.4747
Epoch 7/25
511/511 [==============================] - 263s 515ms/step - loss: 0.4814 - accuracy: 0.7997 - val_loss: 4.4016 - 

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print('Test accuracy :', accuracy)

48/48 [==============================] - 666s 14s/step - loss: 2.1560 - accuracy: 0.3366
Test accuracy : 0.3366336524486542


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score
from tensorflow.keras.utils import to_categorical
import numpy as np

# First, we need to gather the true labels and predicted labels
y_true = test_generator.classes
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Print classification report
report = classification_report(y_true, y_pred_classes, target_names=['Normal', 'Pneumonia', 'Covid-19'])
print(report)

# Calculate AUC-ROC
# AUC-ROC is typically used for binary classification. For multi-class,
# we calculate the average AUC-ROC of each class versus the rest
roc_auc = roc_auc_score(to_categorical(y_true), y_pred, multi_class='ovr')
print('AUC-ROC:', roc_auc)

48/48 [==============================] - 23s 429ms/step
              precision    recall  f1-score   support

      Normal       0.00      0.00      0.00       505
   Pneumonia       0.35      0.47      0.40       505
    Covid-19       0.33      0.54      0.41       505

    accuracy                           0.34      1515
   macro avg       0.23      0.34      0.27      1515
weighted avg       0.23      0.34      0.27      1515

AUC-ROC: 0.5159592196843447


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
